In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers
import spektral
import os
import scipy.sparse as sp

In [2]:
level = 95
bus = 73
samples = 20000
Bus_data = loadtxt('BusData.dat')
Gen_data = loadtxt('GenData.dat')
Bra_data = loadtxt('BranchData.dat')

PD = loadtxt(('Pd_Profile_%d.csv' %bus), delimiter=',')   #Load Profile
PF = loadtxt(('PF_Profile_%d.csv' %bus), delimiter=',')   #Branch Profile
Pg = loadtxt(('Pg_Profile_%d.csv' %bus), delimiter=',')   #Generation Profile
TC = loadtxt(('TC_Profile_%d.csv' %bus), delimiter=',')   #Total Cost 

branch = Bra_data.shape[0]

In [3]:
#Normalize each features to within the range between 0 and 1 (Don't use)
def norm(data):
    max_val = np.amax(data,1,keepdims=1)
    normalize = np.divide(data,max_val)
    return normalize

In [4]:
# # Sum up the total amount of generation per bus
# Pg_Bus = np.zeros([samples,bus])
# for i in range(bus):
#     index = np.nonzero(Gen_data[:,2] == Bus_data[i,1])
#     Pg_Bus[:,i] = np.squeeze(np.sum(Pg[:,index],axis = 2))


# Sum up the total amount of generation capacity and cost per bus
Pg_Cap = np.zeros([bus,2])
for i in range(bus):
    index = np.nonzero(Gen_data[:,2] == Bus_data[i,1])
    Pg_Cap[i] = np.sum(np.squeeze(Gen_data[index,3:5]),axis = 0)

In [5]:
# Create label for graph

Label = np.zeros([samples,PF.shape[1]])
PF_Percent = PF / Bra_data[:,5]

# Label each branch as either 1 or 0 based on flow condition
Label = (np.abs(PF_Percent) >= level/100 * 1)


In [6]:
# np.all([(np.abs(PF_Percent) <= 0.95),(np.abs(PF_Percent) > 0.75)], axis = 0) * 2 + \
# (np.abs(PF_Percent) > 0.95) * 3
# np.all([(np.abs(PF_Percent) <= 0.95),(np.abs(PF_Percent) > 0.85)], axis = 0) * 3 + \



# Label each branch as combination of 0, 0, 0, 0
# Label[:,:,0] = (np.abs(PF_Percent) <= 0.75 ) * 1 
# Label[:,:,1] = np.all([(np.abs(PF_Percent) <= 0.85),(np.abs(PF_Percent) > 0.75)], axis = 0) * 1 
# Label[:,:,2] = np.all([(np.abs(PF_Percent) <= 0.95),(np.abs(PF_Percent) > 0.85)], axis = 0) * 1 
# Label[:,:,3] = (np.abs(PF_Percent) > 0.95) * 1

In [7]:
# Create Edge features
EF = np.zeros([samples,branch,3])
EF[:,:,0] = Bra_data[:,4]        #Reactant
EF[:,:,1] = Bra_data[:,5]        #Line Limit
EF[:,:,2] = Bra_data[:,6]        #Number of parallel lines
EF = EF

# Create Node features
NF = np.zeros([samples,bus,5])
NF[:,:,0]   = PD              #Load Profile
NF[:,:,1:3] = Pg_Cap          #Generator Profile
node = np.zeros([bus])
for i in range(bus):
    node[i] = np.sum( Bra_data[:,2]-1 == i) + np.sum( Bra_data[:,3]-1 == i)
NF[:,:,3] = node              #Number of branches connect to each node
NF[:,:,4] = Bus_data[:,3]     #Bus Type
# NF[:,:,1]   = Pg_Bus
NF = NF


# Create Adjacency matrix
# AM = np.zeros([bus,bus])
AM = sp.csr_matrix((np.ones(branch), (Bra_data[:,2]-1, Bra_data[:,3]-1)), shape = [bus,bus]).toarray()# + sp.csr_matrix((np.ones(branch), (Bra_data[:,3]-1, Bra_data[:,2]-1)), shape = [bus,bus])sp.csr_matrix((np.ones(branch), (Bra_data[:,2]-1, Bra_data[:,3]-1)), shape = [bus,bus]).toarray()

# AM = np.zeros([samples,branch,3])
# AM[:,:,0] = Bra_data[:,2]-1        #Branch From
# AM[:,:,1] = Bra_data[:,3]-1        #Branch To
# AM[:,:,2] = np.ones(branch)      #Weight of each branch

In [8]:
# Create graph files and save them into the folder GNN_Graph
path = 'GNN_Graph'

for i in range(samples):
    filename = os.path.join(path,f'GNN_{i}')
    np.savez_compressed(filename, x = NF[i,:,:], a = AM, e = EF[i,:,:], y = Label[i,:]+1)

In [9]:
# # Code from down here is for DGL


# import dgl
# import torch
# from dgl.data import DGLDataset

In [10]:
# #Load data in to graph dataset for DGL

# class GNNDataset(DGLDataset):
#     def __init__(self):
#         super().__init__(name='OPF')

#     def process(self):
#         self.graphs = []

#         node_features = torch.Tensor(NF)
#         edge_labels   = torch.Tensor(Label)
#         edge_features = torch.Tensor(EF)
#         edges_src     = torch.from_numpy(np.short(Bra_data[:,2]-1))
#         edges_dst     = torch.from_numpy(np.short(Bra_data[:,3]-1))
        
#         for i in range(node_features.shape[0]):
#             g = dgl.graph((edges_src, edges_dst), num_nodes=node_features.shape[1])
#             g.ndata['feat']  = node_features[i,:,:]
#             g.edata['label'] = edge_labels[i]
#             g.edata['feat']  = edge_features[i,:,:]
            
            
#             self.graphs.append(g)
            
#             # Load and save graph
#             # dgl.load_graphs('graph.dgl')
#             # dgl.save_graphs('GNN.dgl', g)
            

#     def __getitem__(self,i):
#         return self.graphs[i]

#     def __len__(self):
#         return len(self.graphs)

In [11]:
# dataset = GNNDataset()


In [12]:
# print(dataset)
# from dgl.dataloading import GraphDataLoader
# dataloader = dgl.dataloading.GraphDataLoader(dataset, batch_size=500, shuffle=True, drop_last=False, num_workers=4)

In [13]:
# from dgl.data import CiteseerGraphDataset 

# dataset2 = CiteseerGraphDataset()

In [14]:
# from dgl.data import SSTDataset
# dataset3 = SSTDataset()

In [15]:
# print(dataset3)

In [16]:
# import dgl.nn as dglnn
# import torch.nn as nn
# import torch.nn.functional as F
# class SAGE(nn.Module):
#     def __init__(self, in_feats, hid_feats, out_feats):
#         super().__init__()
#         self.conv1 = dglnn.SAGEConv(
#             in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
#         self.conv2 = dglnn.SAGEConv(
#             in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

#     def forward(self, graph, inputs):
#         # inputs are features of nodes
#         h = self.conv1(graph, inputs)
#         h = F.relu(h)
#         h = self.conv2(graph, h)
#         return h


  

        
# class MLPPredictor(nn.Module):
#     def __init__(self, in_features, out_classes):
#         super().__init__()
#         self.W = nn.Linear(in_features * 2, out_classes)

#     def apply_edges(self, edges):
#         h_u = edges.src['h']
#         h_v = edges.dst['h']
#         score = self.W(torch.cat([h_u, h_v], 1))
#         return {'score': score}

#     def forward(self, graph, h):
#         # h contains the node representations computed from the GNN defined
#         # in the node classification section (Section 5.1).
#         with graph.local_scope():
#             graph.ndata['h'] = h
#             graph.apply_edges(self.apply_edges)
#             return graph.edata['score']        
        

        

# class Model(nn.Module):
#     def __init__(self, in_features, hidden_features, out_features):
#         super().__init__()
#         self.sage = SAGE(in_features, hidden_features, out_features)
#         self.pred = MLPPredictor()
#     def forward(self, g, x):
#         h = self.sage(g, x)
#         return self.pred(g, h)        

In [17]:
# def evaluate(model, graph, features, labels, mask):
#     model.eval()
#     with torch.no_grad():
#         logits = model(graph, features)
#         logits = logits[mask]
#         labels = labels[mask]
#         _, indices = torch.max(logits, dim=1)
#         correct = torch.sum(indices == labels)
#         return correct.item() * 1.0 / len(labels)

In [18]:
# model = SAGE(in_feats=n_features, hid_feats=100, out_feats=n_labels)
# opt = torch.optim.Adam(model.parameters())

# for epoch in range(10):
#     model.train()
#     # forward propagation by using all nodes
#     logits = model(graph, node_features)
#     # compute loss
#     loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
#     # compute validation accuracy
#     acc = evaluate(model, graph, node_features, node_labels, valid_mask)
#     # backward propagation
#     opt.zero_grad()
#     loss.backward()
#     opt.step()
#     print(loss.item())

In [19]:
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.layers import Dropout, Input
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.regularizers import l2

# from spektral.data.loaders import SingleLoader
# from spektral.datasets.citation import Citation
# from spektral.layers import ARMAConv
# from spektral.transforms import LayerPreprocess

# dataset = Citation("cora", transforms=[LayerPreprocess(ARMAConv)])